In [96]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

df = pd.read_csv ('https://raw.githubusercontent.com/ITACADEMYprojectes/projecteML/refs/heads/main/bank_dataset.CSV')


Tractem el dataset per a deixar-lo al mateix punt que finalitzant la entrega anterior


In [97]:
from scipy.stats.mstats import winsorize

# Tractament dels Outliers Balance
Q1 = df["balance"].quantile(0.25)
Q3 = df["balance"].quantile(0.75)
iqr = Q3 - Q1 # Rang interquartil·lic
upper = Q3 + (1.5*iqr)
lower = Q1 + (1.5*iqr)
outliers = df[(df['balance'] < lower) | (df['balance'] > upper)]
df["balance"]= winsorize(df["balance"], limits=[0.1, 0.1], inplace=True)

# Tractament dels Outliers Age
Q1 = df['age'].quantile(0.05)
Q3 = df['age'].quantile(0.05)
iqr = Q3 - Q1# Rang interquartil·lic
upper = Q3 + (1.5*iqr)
lower = Q1 + (1.5*iqr)
outliers = df[(df['age'] < lower) | (df['age'] > upper)]
df["age"]= winsorize(df["age"], limits=[0.05, 0.05], inplace=True)

#completem els valors categorics desconeguts amb "unknown"
df['marital'] = df["marital"].fillna("unknown")
df['education'] = df["education"].fillna("unknown")

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11162 entries, 0 to 11161
Data columns (total 17 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   age        11162 non-null  float64
 1   job        11162 non-null  object 
 2   marital    11162 non-null  object 
 3   education  11162 non-null  object 
 4   default    11162 non-null  object 
 5   balance    11162 non-null  int64  
 6   housing    11162 non-null  object 
 7   loan       11162 non-null  object 
 8   contact    11162 non-null  object 
 9   day        11162 non-null  int64  
 10  month      11162 non-null  object 
 11  duration   11162 non-null  int64  
 12  campaign   11162 non-null  int64  
 13  pdays      11162 non-null  int64  
 14  previous   11162 non-null  int64  
 15  poutcome   11162 non-null  object 
 16  deposit    11162 non-null  object 
dtypes: float64(1), int64(6), object(10)
memory usage: 1.4+ MB


In [98]:
#reduim la dimensionalitat en totes les variable categoriques

#reduim les dimensions de job
valor_others = df['job'].isin(['unemployed', 'unknown', 'housemaid', 'entrepreneur'])
df.loc[valor_others, 'job'] = 'others'
df['job'] = df['job'].astype('category')

#reduim les dimensions de marital
valor_others = df['marital'].isin(['divorced', 'unknown'])
df.loc[valor_others, 'marital'] = 'others'
df['marital']=df['marital'].astype('category')

#reduim les dimensions de education
valor_others = df['education'].isin(['unknown','primary'])
df.loc[valor_others, 'education'] = 'others'
df['education']=df['education'].astype('category')

default_={'yes':1,'no':0}
df['default']=df['default'].map(default_)

housing_={'yes':1,'no':0}
df['housing']=df['housing'].map(housing_)

loan_={'yes':1,'no':0}
df['loan']=df['loan'].map(loan_)

#reduim les dimensions de contact
valor_others = df['contact'].isin(['unknown','telephone'])
df.loc[valor_others, 'contact'] = 'others'

#reduir les dimensions de poutcome
valor_others = df['poutcome'].isin(['unknown','other'])
df.loc[valor_others, 'poutcome'] = 'others'

deposit_={'yes':1,'no':0}
df['deposit']=df['deposit'].map(deposit_)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11162 entries, 0 to 11161
Data columns (total 17 columns):
 #   Column     Non-Null Count  Dtype   
---  ------     --------------  -----   
 0   age        11162 non-null  float64 
 1   job        11162 non-null  category
 2   marital    11162 non-null  category
 3   education  11162 non-null  category
 4   default    11162 non-null  int64   
 5   balance    11162 non-null  int64   
 6   housing    11162 non-null  int64   
 7   loan       11162 non-null  int64   
 8   contact    11162 non-null  object  
 9   day        11162 non-null  int64   
 10  month      11162 non-null  object  
 11  duration   11162 non-null  int64   
 12  campaign   11162 non-null  int64   
 13  pdays      11162 non-null  int64   
 14  previous   11162 non-null  int64   
 15  poutcome   11162 non-null  object  
 16  deposit    11162 non-null  int64   
dtypes: category(3), float64(1), int64(10), object(3)
memory usage: 1.2+ MB


In [99]:
# Codificacio Categorica One-Hot
from sklearn.preprocessing import OneHotEncoder

codificador = OneHotEncoder()
codificacion = codificador.fit_transform(df[['job', 'marital','education','contact','month','poutcome']])

new_columns = pd.DataFrame(codificacion.toarray(),
                    columns=codificador.get_feature_names_out())

combined_df = pd.concat([df, new_columns], axis=1)
df = combined_df.drop(['job', 'marital','education','contact','month','poutcome'], axis=1)
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11162 entries, 0 to 11161
Data columns (total 43 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   age                  11162 non-null  float64
 1   default              11162 non-null  int64  
 2   balance              11162 non-null  int64  
 3   housing              11162 non-null  int64  
 4   loan                 11162 non-null  int64  
 5   day                  11162 non-null  int64  
 6   duration             11162 non-null  int64  
 7   campaign             11162 non-null  int64  
 8   pdays                11162 non-null  int64  
 9   previous             11162 non-null  int64  
 10  deposit              11162 non-null  int64  
 11  job_admin.           11162 non-null  float64
 12  job_blue-collar      11162 non-null  float64
 13  job_management       11162 non-null  float64
 14  job_others           11162 non-null  float64
 15  job_retired          11162 non-null 

Sel·leccionar subconjunts de dades d'entrenament i de prova

In [100]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

Y=df.loc[:,['deposit']]
X=df.loc[:, df.columns != 'deposit']

# Escalem el conjunt de les dades X
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Apliquem PCA
pca = PCA(n_components=0.90)
X_pca = pca.fit_transform(X_scaled)


X_train, X_test, Y_train, Y_test = train_test_split(X_pca, Y,
                                                    test_size=0.2, # 80% assignat a entrenament + 20% assignat a test
                                                    random_state=0, # seleccionar dades aleatoriament
                                                    stratify=Y) # mantenir la mateixa distribucio èxit/fracàs en ambdos conjunts

X.shape, X_train.shape, X_test.shape,Y_train.shape, Y_test.shape

((11162, 42), (8929, 29), (2233, 29), (8929, 1), (2233, 1))

##Conclusio:


*   Hem reduit la dimensionalitat de les variables amb mes valors possibles
*   Hem escalat la matriu de categories
* Hem aplicat PCA per passar de 42 columnes a 29 mantenint una variança del 90%
* Finalment hem dividit les dades en dos conjunts de prova (2233 - 20%) i entrenament (8929 - 80%), mantenint la mateixa ratio d'exit/fracas en ambdos conjunts.


